In [1]:
import os, os.path
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf_device='/gpu:0'
is_pred =  False #True #

args = {
    'lr':1e-4,
    'exp_steps':9000,
    'train_st':1000,
    'memory_len':1000,
    'update_target_rate':1000,
    'net':'DDQN_CNNDNN',
    'batch_size': 32
}

num_episode = 10000
global_step = 0
tr_l, h_fill, tr_r,avg_loss_l,history_eps,used_boxes_eps  = [],[],[],[],[],[]
# K = 1
max_k = 1
K = 1
n_candidates = 200
num_max_remain = 180 #
num_bbox_type = 20 #사용할 대박스 수
sc_target = 64 #최대 대박스 크기를 sc_target 크기만큼 줄임
n_channel = 4

mi = 'U1'
mbox_type = 'PP'
f_name = 'plan_predict_input/' + mi+'_'+mbox_type
output_col = ['pack_no','creat_dd', 'odr_no', 'ctmmny_cd','vctp_cd', 'prod_no', 'mbox_cd', 'bbox_cd',
              'B_CBM',  'mbox_acctc_qty', 'bbox_acctc_qty', 
              'packing_rate_volume', 'packing_rate_cbm','x','y','z']

In [ ]:
class DQNAgent_bbox:
    def __init__(self, L=20, B=20, H=20, n_remains = 5, n_loading=3, n_channel = 6, max_size = 64,
                 lr=1e-8, batch_size = 32, exp_steps=500, train_st = 200, memory_len=500, update_target_rate = 30, net='DNN' ):
        self.state_size = (L, B, 3)
        self.selected_size = (L, B, n_loading)
        self.remain_size = (n_remains, max_size ,n_channel) #(L, B, n_remains)
        self.loading_size = (n_loading, max_size ,n_channel//2) # (L, B, n_loading)
        self.cbm_size = (1 + n_remains*2 + n_loading,)
        self.output_size = 1 #math.factorial(c_boxes_size)
        # hyperparameters
        self.discount_factor = 0.99
        self.learning_rate = lr#1e-8#1e-4
        self.epsilon = 1.
        self.epsilon_start, self.epsilon_end = 1.0, 0.01
        self.exploration_steps = exp_steps
        self.epsilon_decay_step = self.epsilon_start - self.epsilon_end
        self.epsilon_decay_step /= self.exploration_steps
        self.batch_size = batch_size
        self.train_start = train_st
        self.update_target_rate = update_target_rate
        self.beta = 0.2
        self.memory = deque(maxlen=memory_len) # replay memory
        self.net = net
        self.vmin = 0#-0.02
        self.vmax = 1
        self.nsup = 51#52
        self.dz = (self.vmax - self.vmin)/(self.nsup - 1.)
        self.z = np.linspace(self.vmin,self.vmax,self.nsup)
        self.gamma = 0.9
        self.criterion = tf.keras.losses.CategoricalCrossentropy()
        self.dist = False
        # model
        if net == 'DNN':
            self.model = DQN_DNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.output_size)
            self.target_model = DQN_DNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.output_size)
        elif net =='CNN':
            self.model = DQN_CNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.output_size)
            self.target_model = DQN_CNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.output_size)
        elif net =='CNNDNN':
            self.model = DQN_CNNDNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.output_size)
            self.target_model = DQN_CNNDNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.output_size)
        # distributed q learning
        elif net == 'DDQN_DNN':
            self.model = DQN_DNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.cbm_size, self.nsup)
            self.target_model = DQN_DNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.cbm_size, self.nsup)
        elif net == 'DDQN_CNNDNN':
            self.model = DQN_CNNDNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.cbm_size, self.nsup)
            self.target_model = DQN_CNNDNN(self.state_size, self.selected_size, self.remain_size, self.loading_size, self.cbm_size, self.nsup)
        
        if net in ['DDQN_DNN', 'DDQN_CNNDNN']:
            print('distribution', net)
            self.dist = True
        
        self.optimizer = Adam(self.learning_rate)#, clipnorm=10.)
        # target model (init)
        self.update_target_model()
        self.avg_q_max, self.avg_loss = 0, 0
#         self.writer = tf.summary.create_file_writer('summary/bpp')
        self.model_path = os.path.join(os.getcwd(), 'save_model', 'model_3d')
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        
    def get_action(self, state, loaded_mh_c, r_boxes, loading, cbm):
        if np.random.rand() <= self.epsilon:
            random_action = random.randrange(len(state))
            return random_action
        else:
            if self.dist:
                z = self.model.predict([state, loaded_mh_c, r_boxes, loading, cbm]) #(C,51)
                z_concat = np.vstack(z) #action output이 여러개 일때
                #q = np.sum(np.multiply(z_concat[:,45:], np.array(self.z)[45:]), axis=1) #(C,)
                q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1) #(C,)
                self.avg_q_max += np.amax(q)
                return np.argmax(q)
            else:
                q_values=self.model([state, loaded_mh_c, r_boxes, loading, cbm])
                armax_idx = np.where( q_values == tf.math.reduce_max(q_values))
                action_idx = armax_idx[0][0]
                return action_idx
            
    def append_sample(self, history, load, remain_size, load_size, reward, last, t_history, t_load, t_remain_size, t_load_size, cbm,  t_cbm):
        self.memory.append(( history, load, remain_size, load_size, reward, last, t_history, t_load, t_remain_size, t_load_size, cbm, t_cbm))
        
        
#     def draw_tensorboard(self, reward, fill, step, episode):
#         with self.writer.as_default():
#             tf.summary.scalar('Total Reward/Episode', reward, step=episode)
#             tf.summary.scalar('Total Fill/Episode', fill, step=episode)
#             tf.summary.scalar('Average Max Q/Episode', self.avg_q_max / float(step), step=episode)
#             tf.summary.scalar('Duration/Episode', step, step=episode)
#             tf.summary.scalar('Average Loss/Episode', self.avg_loss / float(step), step=episode)

    def train_model(self):
        
        batch = random.sample(self.memory, self.batch_size)
        
        history = np.array([sample[0] for sample in batch])# (B, 20,20,2)
        load = np.array([sample[1] for sample in batch])# (B, 20,20,1)
        remain_size = np.array([sample[2] for sample in batch]) # (B, 20,20, max_num_remain)
        load_size = np.array([sample[3] for sample in batch]) # (B, 20, 20, K)
        reward = np.array([sample[4] for sample in batch]) # (B,)
        dones = np.array([sample[5] for sample in batch]) # (B,)
        len_t_comb = [ len(sample[6]) for sample in batch ]
        t_history = np.concatenate([sample[6] for sample in batch] )
        t_load = np.concatenate([sample[7] for sample in batch])
        t_remain_size = np.concatenate([sample[8] for sample in batch] )
        t_load_size = np.concatenate([sample[9] for sample in batch] )
        cbm = np.array([sample[10] for sample in batch] )
        t_cbm = np.concatenate([sample[11] for sample in batch] )
        #print(history.shape,load.shape,remain_size.shape, load_size.shape, reward.shape, dones.shape,  t_history.shape, t_load.shape, t_remain_size.shape, t_load_size.shape)
        
        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            if self.dist:
                # 예측
                predicts = self.model([history, load, remain_size, load_size, cbm]) # B,51
                # 타겟
#                 z_b = []
#                 for i in range(0,len(t_history), self.batch_size):
#                     e_idx = min( i + self.batch_size, len(t_history) )
#                     z = self.target_model([t_history[i:e_idx], t_load[i:e_idx], t_remain_size[i:e_idx], t_load_size[i:e_idx]]) #(C, 51)
#                     z_b.append(z)
#                 z_concat = np.vstack(z_b) #(B*c, 51)
                z_concat = self.target_model([t_history, t_load, t_remain_size, t_load_size, t_cbm])
                t_q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1) #(B*c, )
                # 가장 기댓값이 높은 action 선택
                next_actions, probs = [],[]
                for i in range(len(len_t_comb)):
                    s_idx = np.sum(len_t_comb[:i]).astype('int')
                    if len_t_comb[i] ==1:
                        n_a = 0
                    else:                        
                        n_a = np.argmax(t_q[s_idx: s_idx + len_t_comb[i]])
                    next_actions.append(n_a) #[32]
                    probs.append((z_concat[s_idx: s_idx + len_t_comb[i]])[n_a])
                probs = np.stack(probs) #(B, 51)
                # target distribution
                bj = np.round((reward - self.vmin)/self.dz) #(B,)
                #지지 성분의 색인
                bj = np.clip(bj, 0, self.nsup-1).astype('int') #유효한 범위
                targets = []
                for i in range(self.batch_size):
                    if dones[i]:
                        target_dist = np.zeros(self.nsup)
                        target_dist[bj[i]] = 1
                        targets.append(target_dist)
                    else:
                        m = probs[i].copy() #(51,)
                        j = 1
                        for i in range(bj[i],1,-1):
                            m[i] += np.power(self.gamma, j) * m[i-1]
                            j += 1
                        j = 1
                        for i in range(bj[i], self.nsup-1,1):
                            m[i] += np.power(self.gamma, j) * m[i+1]
                            j += 1
                        m /= m.sum() #(51,)
                        targets.append(m)
                loss = self.criterion(targets, predicts)
            else: 
                # 예측
                predicts = self.model([history, load, remain_size, load_size]) #(B, 1)
                # 타겟
                targets = []
                for i in range(self.batch_size):
                    t_q = self.target_model([t_history[i], t_load[i], t_remain_size[i], t_load_size[i], t_cbm[i]]) # target q value
                    t_max_q = tf.math.reduce_max(t_q)
                    targets.append([(1- 0.75)*reward[i] + (1 - dones[i]) *0.75*t_max_q])
                targets=np.array(targets) #(B, 1)
                # loss 계산
                #loss = tf.reduce_mean(tf.square(targets - predicts))
                error = tf.abs(targets - predicts)
                quadratic_part = tf.clip_by_value(error, 0.0, 1.0)
                linear_part = error - quadratic_part
                loss = tf.reduce_mean(0.5 * tf.square(quadratic_part) + linear_part)
            
            self.avg_loss += loss.numpy()
            
        # update
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))

In [2]:
import random
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.keras.backend.floatx()
from collections import deque
import itertools
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from libs.utils import *
from libs.generate_boxes import *
from libs.dqn import *


# 대박스 규격 로드
df_bbox = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
casemaster=pd.read_csv('data/preprocessed/casemaster.csv')
df_bbox = df_bbox.iloc[:num_bbox_type]
df_bbox = pd.merge(df_bbox, casemaster[['CD', 'CBM']], how='left', left_on=['CASE_CD'], right_on=['CD']).drop(['CD'],axis=1)
df_bbox = df_bbox.drop(['CASE_CD_count'], axis=1)
# 대박스 규격 로드 (Env)
bbox_size = pd.read_csv('data/preprocessed/bbox/' + mi + '_' + mbox_type + '.csv')
bbox_size = bbox_size.values[:num_bbox_type, 1:-1]
bbox_size = np.concatenate([np.sort(bbox_size[:,:2], axis=1), bbox_size[:,2:]], axis=-1) # 정렬
# 대박스 스케일링
sc = sc_target/np.max(bbox_size, axis =0)[:2] 
bbox_sc = bbox_size.copy() 
bbox_sc[:, :2] = bbox_size[:,:2] * sc # 세로가로만 스케일링
bbox_sc = (bbox_sc+0.5).astype('int') #반올림
mxl, mxb, mxh = np.max(bbox_sc, axis =0 ) # 최대 규격
print(mxl, mxb, mxh)
# 대박스 행렬로 변환
bbox_type = [] #np.zeros((1, 20, 20))
for l,b,h in bbox_sc:
    bbox = np.zeros((mxl, mxb))
    bbox[l:,:] = mxh
    bbox[:,b:] = mxh
    bbox[:l,:b] = mxh - h
    bbox_type.append(bbox)
bbox_type = np.stack((bbox_type)) #(5, 20, 20)
# 대박스 CBM 스케일링
bbox_cbm_mx = np.max(df_bbox['CBM'])

# 중박스 데이터 로드 (data frame)
group_key = ['creat_dd', 'odr_no', 'ctmmny_cd', 'mbox_type']
resid = pd.read_csv('data/preprocessed/'+f_name +'_resid.csv')
org_col = resid.columns
resid = resid.sort_values('sum_wt', ascending = False)
resid = resid.set_index(group_key).sort_index()
group_idx = resid.index.drop_duplicates()
# 중박스 데이터 로드 (입력 데이터)
with open('data/preprocessed/' + f_name +'_resid.pickle', 'rb') as handle: 
    boxes_multi =  pickle.load(handle)
with open('data/preprocessed/' + f_name +'_resid_v0.pickle', 'rb') as handle: 
    boxes_multi_n =  pickle.load(handle)
# 중박스 정렬
boxes_multi = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi]
boxes_multi_n = [ np.concatenate([np.sort(x[:, :2], axis=1), x[:, 2:]], axis=-1) for x in  boxes_multi_n]
# 중박스 크기 스케일링
boxes_multi_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, 2:3]], axis=-1).astype('int') for x in  boxes_multi]
boxes_multi_n_sc = [ np.concatenate([np.ceil(x[:, :2] * sc), x[:, 2:6]], axis=-1) for x in  boxes_multi_n]
# CBM
cbm_bbox = df_bbox['CBM'].values # 대박스 CBM
cbm_bbox = (cbm_bbox - np.min(cbm_bbox)) / np.ptp(cbm_bbox) # 스케일링된 대박스 CBM
cbm_per_mbox = [x[:,-1] for x in boxes_multi] # 중박스 CBM
cbm_mx = np.max(np.concatenate(cbm_per_mbox))
cbm_mn = np.min(np.concatenate(cbm_per_mbox))
cbm_mbox = [ x/cbm_mx for x in cbm_per_mbox] #[ (x-cbm_mn)/(cbm_mx - cbm_mn) for x in cbm_per_mbox] # 스케일링된 중박스 CBM
# 중박스 크기를 행렬로 표현 & 범위 스케일링
mbox_mat = [x.copy() for x in boxes_multi_n_sc] #.copy -> not working
#for i in range(len(mbox_mat)): mbox_mat[i][:,2] =  mbox_mat[i][:,2]/mxh #높이 스케일링
for i in range(len(mbox_mat)): mbox_mat[i][:,2] = mbox_mat[i][:,2] /np.max(resid['M_H'])  #(mbox_mat[i][:,2] - np.min(resid['M_H'])) / (np.max(resid['M_H']) - np.min(resid['M_H']))
# for i in range(len(mbox_mat)): mbox_mat[i][:,4] =  (mbox_mat[i][:,4]-cbm_mn)/(cbm_mx-cbm_mn) #cbm 스케일링
# for i in range(4: print(np.min(np.concatenate(mbox_mat)[..., i]), np.max(np.concatenate(mbox_mat)[..., i]))
max_size = sc_target
mbox_mat = np.stack([size2mat(x, num_max_remain, max_size) for x in mbox_mat]) # (N, num_max_remain, max_size, 4)
mbox_mat = np.concatenate([mbox_mat, np.zeros_like(mbox_mat)], axis = -1) # 남은 박스 규격 + 적입된 박스 규격

# Environment
env = Bpp3DEnvMS(length = mxl, breadth = mxb, height = mxh, bbox_type = bbox_type)
# Agent
agent = DQNAgent( L = mxl, B = mxb , H = mxh, n_remains = num_max_remain, n_loading=K, n_channel = n_channel, max_size = max_size,
                 lr=args['lr'], batch_size = args['batch_size'], exp_steps=args['exp_steps'], train_st = args['train_st'], 
                 memory_len=args['memory_len'], update_target_rate = args['update_target_rate'], net = args['net']) #DDQN_CNNDNN

bbox_agent = DQNAgent_bbox( L = mxl, B = mxb , H = mxh, n_remains = num_max_remain, n_loading=K, n_channel = n_channel, max_size = max_size,
                 lr=args['lr'], batch_size = args['batch_size'], exp_steps=args['exp_steps'], train_st = args['train_st'], 
                 memory_len=args['memory_len'], update_target_rate = args['update_target_rate'], net = args['net']) #DDQN_CNNDNN

64 64 1320
distribution DDQN_CNNDNN


In [ ]:
if is_pred:
    num_episode = len(boxes_multi_sc)
    # 예측 결과 테이블
    pk = pd.DataFrame(columns = ['pack_no']+list(org_col)+['packing_rate_cbm','packing_rate_volume','x','y','z'])
    pack_no = 0
    agent.epsilon = 0
    #agent.model.load_weights('save_model/model')
    
for e in range(num_episode): #
    st=time.time()
    step = 0
    ith_data = e % len(boxes_multi) # 4# 5#
    
    if is_pred:
        df_p = resid.loc[group_idx[ith_data]].reset_index().copy()
        df_p = df_p.loc[df_p.index.repeat(df_p['num_mbox'])] # 각 행을 num_mbox번 만큼 반복
        
    else:
        if e > 9000:
            agent.epsilon = 0
            bbox_agent.epsilon = 0
        if agent.epsilon > agent.epsilon_end and len(agent.memory)>=agent.train_start:
            agent.epsilon -= agent.epsilon_decay_step
        if bbox_agent.epsilon > bbox_agent.epsilon_end and len(bbox_agent.memory)>=bbox_agent.train_start:
            bbox_agent.epsilon -= bbox_agent.epsilon_decay_step
           
    boxes_all_sc = boxes_multi_sc[ ith_data].copy().astype('int')# 
    cbm_all = cbm_per_mbox[ith_data].copy()
    r_boxes = boxes_all_sc.copy()
    m_mat = mbox_mat[ith_data].copy() #(50,64,2)
    boxes_idx = np.arange(len(boxes_all_sc))
    loaded_idx = []
    used_boxes, pred_pos = [],[]
    env_rewards, is_last, used_bbox = [], [], []
    history, h_load, h_remain_size, h_load_size, h_cbm = [],[],[],[],[]
    t_history, t_load, t_remain_size, t_load_size, t_cbm = [],[],[],[],[]
    
    while len(r_boxes)>0: #모든 중박스를 다 적입할 때까지 반복 (새로운 대박스 시작)
        done = False
        len_group = 0 #현재 대박스에 적입된 중박스 수
        cbm_reward = 0
        # state 선택
        mask = [ sum((i >= r_boxes[:n_candidates]).sum(1)==3)>=1 for i in bbox_sc] #적입 가능한 대박스중에서만 선택
        if sum(mask)==0:
            r_boxes_rot = np.stack([r_boxes[:n_candidates,1],r_boxes[:n_candidates,0],r_boxes[:n_candidates,2]], axis = -1)
            mask = [ sum((i >= r_boxes_rot).sum(1)==3)>=1 for i in bbox_sc]
        in_state = np.array( [ np.stack([b, b, b], axis=-1) for b in bbox_type[mask]] )
        in_state = (in_state/mxh).astype(np.float32)
        loading_loc_c = np.zeros((sum(mask), mxl,mxb, K)) # 적재 위치
        in_loading = np.zeros((sum(mask), K, max_size, n_channel//2))
        in_remain = np.array([m_mat]*20).copy()
        #for i in range(20): in_remain[i,...,[2,5]] = in_remain[i,...,[2,5]]/df_bbox['CBM'].values[i]       
        in_remain = in_remain[mask] 
        in_cbm = init_cbm(cbm_bbox[mask],bbox_cbm_mx, cbm_mbox[ith_data], boxes_idx, num_max_remain, K) #l
        # 대박스 규격 선택
        action_idx = bbox_agent.get_action(in_state, loading_loc_c, in_remain, in_loading, in_cbm)
        bbox_idx = np.where(mask)[0][action_idx]
        env.reset(bbox_idx)
        history.append(in_state[action_idx])
        h_load.append(loading_loc_c[action_idx])
        h_remain_size.append(in_remain[action_idx])
        h_load_size.append(in_loading[action_idx])
        h_cbm.append(in_cbm[action_idx])
        s_bbox_cbm = df_bbox.iloc[env.bbox_idx]['CBM']
        
        while not done:#중박스에 대해 반복##############
            #print('m_mat 1',np.sum(m_mat[:,:,0]),np.sum(m_mat[:,:,1]),np.sum(m_mat[:,:,2]),np.sum(m_mat[:,:,3]))
            state = env.container_s.copy()
            state_h = env.container_h.copy()
            global_step += 1
            step += 1
            # CBM
            cbm_all_ratio = cbm_all/s_bbox_cbm
            # 적재 위치 계산 ########################
            k = 1 #min(K, len(r_boxes))
            selected, selected_idx = cbn_select_boxes(r_boxes[:n_candidates],boxes_idx[:n_candidates], k) # 조합
            s_order, s_order_idx = get_selected_order(selected, selected_idx, k) #순열
            #s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
            #s_order, s_order_idx = merge_order_rep(s_order, s_order_idx, selected, r_boxes ,boxes_idx, max_k)
            add_skip = False #if len_group == 0 else True #add_skip=False: 적입하지 않는 action은 없음
            order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
                get_selected_location(s_order, s_order_idx, state, state_h, env.height,K, cbm_reward ,cbm_all_ratio,add_skip ) #위치
            
            # action ########################
            if np.sum(num_loading_c) != 0: #하나도 적입하지 못한 경우(취할 action이 없는 경우)
                in_state, in_remain, in_loading = \
                    raw2input(state, state_h, env.bbox, m_mat, mbox_mat[ith_data], s_bbox_cbm, [s_order_idx[i] for i in order_idx_c], K, mxh)#num_max_remain, K, max_size, 
                in_cbm = get_in_cbm(s_bbox_cbm, bbox_cbm_mx, cbm_mbox[ith_data], boxes_idx, loaded_idx, len_group, [s_order_idx[i] for i in order_idx_c], num_max_remain, K)
                if np.sum(num_loading_c) == 1: action_idx = 0 #가능한 action 수가 1개인 경우
                else: action_idx = agent.get_action(in_state, np.array(loading_loc_c), in_remain, in_loading, in_cbm)
            else:
                action_idx = 0 
                in_state, in_remain, in_loading = \
                    raw2input(state, state_h, env.bbox, m_mat, mbox_mat[ith_data], s_bbox_cbm, [], K, mxh)
                loading_loc_c = np.zeros((1, mxl, mxb, K)) 
                next_state_c =  state[np.newaxis].copy()
                next_h_c = state_h[np.newaxis].copy()
                in_cbm = get_in_cbm(s_bbox_cbm, bbox_cbm_mx, cbm_mbox[ith_data], boxes_idx, loaded_idx, len_group, [], num_max_remain, K)
                
            # 대박스 선택 후의 target network 입력 (첫 번째 중박스가 적입 안되는 경우는 없음)
            if len_group == 0:
                t_history.append(in_state) # 적입 전 상태
                t_load.append(np.array(loading_loc_c)) # 적입된 상태
                t_remain_size.append(in_remain) # 선택 후 전처리된 남은 박스 사이즈
                t_load_size.append(in_loading) # 적입된 박스 사이즈
                is_last.append(False)
                t_cbm.append(in_cbm)
              
            # 다음 step ################################
            if np.sum(num_loading_c) == 0: # 주어진 중박스 중 하나도 적재하지 못한 경우
                r_boxes = get_remain(r_boxes[:n_candidates], r_boxes) # 해당 범위의 박스들 제외
                m_mat = drop_remain(m_mat, boxes_idx[:n_candidates])
                boxes_idx = boxes_idx[n_candidates:]
            else: # 한 가지 이상의 action 옵션이 있는 경우(적재 가능한 경우)
                env.step(next_state_c[action_idx], next_h_c[action_idx])  # env의 다음 state
                # 업데이트
                order_idx = order_idx_c[action_idx]
                loading_idx = loading_idx_c[action_idx]
                len_group += num_loading_c[action_idx] # bbox
                # index 업데이트
                loaded_idx += loading_idx # 적입된 박스 인덱스
                boxes_idx = np.setdiff1d(boxes_idx, s_order_idx[order_idx]) # 남은 박스의 인덱스
                # boxes_idx = np.setdiff1d(boxes_idx, loading_idx) 
                # 남은 박스 업데이트
                r_boxes = boxes_all_sc[boxes_idx]
                m_mat = mbox_mat[ith_data].copy()
                m_mat = add_loading(m_mat, loaded_idx, len_group)
                m_mat = drop_remain(m_mat, loaded_idx)
                # cbm
                cbm_reward += (np.sum(cbm_all_ratio[loading_idx]))#/df_bbox.iloc[env.bbox_idx]['CBM'])
                # 사용된 박스, 위치 업데이트
                used_boxes = used_boxes + loading_size_c[action_idx] #사용된 박스 업데이트
                pred_pos = pred_pos + loading_xyz_c[action_idx] #예측된 위치 업데이트
                
            #남은 박스가 없음 or 높이맵이 모두 채워짐 or cbm 모두 채워짐 or 적입할 수 없음
            if len(r_boxes) == 0 or np.sum(env.container_h != env.height) == 0 or cbm_reward>=1  or np.sum(num_loading_c) == 0:
                done = True
            
            # action 선택 수가 2 이상이거나 done일 경우 -> append
            if len(num_loading_c) > 1 or done:           
                # (s, a) 
                history.append(in_state[action_idx]) # 적입 전 상태
                h_load.append(loading_loc_c[action_idx]) # 적입 위치
                h_remain_size.append(in_remain[action_idx]) # 남은 박스/이전에 적입되어왔던 박스
                h_load_size.append(in_loading[action_idx]) # 적입된 박스 사이즈
                h_cbm.append(in_cbm[action_idx])
                # (s') 
                t_state = next_state_c[action_idx].copy() # 다음 state 
                t_state_h = next_h_c[action_idx].copy()
                if done: #마지막 박스까지 채워졌을 경우 or 더 이상 적입할 수 없을 경우
                    in_t_history = np.stack([t_state, t_state_h, env.bbox],axis=-1).reshape((1, mxl, mxb,3))
                    in_t_history = in_t_history/mxh
                    loading_loc_c = np.zeros((1, mxl, mxb, K)) 
                    in_t_remains = m_mat[np.newaxis].copy()
                    #in_t_remains[...,[2,5]] = in_t_remains[...,[2,5]]/s_bbox_cbm 
                    in_t_loading = np.zeros((1, K, max_size, n_channel//2))
                    in_t_cbm = get_in_cbm(s_bbox_cbm, bbox_cbm_mx, cbm_mbox[ith_data], boxes_idx, loaded_idx, len_group, [], num_max_remain, K)
                else: # 다음 state에 대해 action을 취할 수 있는 경우
                    k = 1 #min(K, len(r_boxes))
                    selected,selected_idx = cbn_select_boxes(r_boxes[:n_candidates], np.arange(len(r_boxes[:n_candidates])), k) # 조합
                    s_order,s_order_idx = get_selected_order(selected, selected_idx, k) # 순열
                    #s_order, s_order_idx = rot_one_order(s_order, s_order_idx) # 회전
                    #s_order, s_order_idx = merge_order_rep(s_order, s_order_idx, selected, r_boxes ,boxes_idx, max_k)
                    order_idx_c, num_loading_c, loading_idx_c, loading_size_c, loading_xyz_c, loading_loc_c, next_state_c, next_h_c  =\
                        get_selected_location(s_order, s_order_idx, t_state, t_state_h, env.height, K, cbm_reward ,cbm_all_ratio )
                    if sum(num_loading_c)==0:
                        loading_loc_c = np.zeros((1, env.length, env.breadth, K))
                        in_t_history, in_t_remains, in_t_loading = \
                            raw2input(t_state, t_state_h, env.bbox, m_mat,mbox_mat[ith_data], s_bbox_cbm, [], K, mxh)
                        in_t_cbm = get_in_cbm(s_bbox_cbm, bbox_cbm_mx, cbm_mbox[ith_data], boxes_idx, loaded_idx, len_group, [], num_max_remain, K)
                    else:
                        in_t_history, in_t_remains, in_t_loading = \
                            raw2input(t_state, t_state_h, env.bbox, m_mat,mbox_mat[ith_data], s_bbox_cbm, [s_order_idx[i] for i in order_idx_c], K, mxh) #num_max_remain, K, max_size, 
                        in_t_cbm = get_in_cbm(s_bbox_cbm, bbox_cbm_mx, cbm_mbox[ith_data], boxes_idx,loaded_idx, len_group,[s_order_idx[i] for i in order_idx_c], num_max_remain, K)
                        
                t_history.append(in_t_history) 
                t_load.append(np.array(loading_loc_c)) 
                t_remain_size.append(in_t_remains)
                t_load_size.append(in_t_loading)
                is_last.append(done) 
                t_cbm.append(in_t_cbm)
            if done:
                env_rewards.append(cbm_reward)
                #env_rewards.append(env.terminal_reward())
            if cbm_reward>1: break   
            # 학습 시작
            if len(agent.memory) >= agent.train_start & ~is_pred:
                agent.train_model()
                if global_step % agent.update_target_rate == 0: # target model 업데이트
                    agent.update_target_model()
                avg_loss_l.append(agent.avg_loss/float(step))
            if len(bbox_agent.memory) >= bbox_agent.train_start & ~is_pred:
                bbox_agent.train_model()
                if global_step % bbox_agent.update_target_rate == 0: # target model 업데이트
                    bbox_agent.update_target_model()
            
            
#             %matplotlib inline
#             sizes = boxes_multi[ ith_data].copy().astype('int')[loaded_idx][:,:3]#boxes_all[loaded_idx][:,:3]
#             positions = np.array(pred_pos.copy())
#             positions[:, :2] = positions[:, :2]/sc
#             vis_box(sizes,positions,fs=(3,3),mn=-5, mx=1500)
            
            #if len(loaded_idx) > 3: break
            ################################################################################ 두 번째 while 끝
            
        #break   
        ################################################################################ 첫 번째 while 시작
        #print('m_mat 2',np.sum(m_mat[:,:,0]),np.sum(m_mat[:,:,1]),np.sum(m_mat[:,:,2]),np.sum(m_mat[:,:,3]))
        boxes_idx = np.setdiff1d(np.arange(len(boxes_all_sc)), loaded_idx)
        r_boxes = boxes_all_sc[boxes_idx]
        if len_group>0: used_bbox.append([env.bbox_idx]*len_group)
        if len_group>0: tr_l.append(cbm_reward)#(env.terminal_reward())
        m_mat = drop_remain(mbox_mat[ith_data], loaded_idx) #적입된 상자는 0으로 초기화
        #print('m_mat 3',np.sum(m_mat[:,:,0]),np.sum(m_mat[:,:,1]),np.sum(m_mat[:,:,2]),np.sum(m_mat[:,:,3]))
    ################################################################################ 
    
    # terminal reward 계산 -> replay memory에 추가 
    if done and len(used_boxes)==len(boxes_all_sc):
        # terminal reward 
        a_repeate = 6 if np.mean(env_rewards) > 0.9 else 1
        # append samples
        N = len(history)
        for i in range(N):
            end_epi = True if i==N-1 else False
            if is_last[i] & ~end_epi: continue # 대박스 적입이 끝나는 것은 학습하지 않음
            len_remain_group = sum(is_last[i:]) # 남은 포장 대박스 수 (5,4,3,...)
            reward = np.mean(env_rewards[-len_remain_group:]) #step reward
            for a in range(a_repeate):
                agent.append_sample(history[i], h_load[i], h_remain_size[i], h_load_size[i], reward, end_epi, #is_last[i],
                                    t_history[i], t_load[i], t_remain_size[i], t_load_size[i], h_cbm[i], t_cbm[i])
                bbox_agent.append_sample(history[i], h_load[i], h_remain_size[i], h_load_size[i], reward, end_epi, #is_last[i],
                                    t_history[i], t_load[i], t_remain_size[i], t_load_size[i], h_cbm[i], t_cbm[i])
                
    log = "=====episode:{:5d}| ".format(e)
    log += "memory length:{:5d}| ".format(len(agent.memory))
    log += "epsilon:{:.3f}| ".format(agent.epsilon)
    log += "reward:{:2d}, {:.3f} (".format(len(env_rewards), np.mean(env_rewards))
    log += ', '.join('{:.3f}'.format(k) for k in env_rewards)
    #log += "t_reward: {:.3f} | ".format(t_reward)
    #log += "q avg : {:3.2f} | ".format(agent.avg_q_max / float(step))
    log += ")| avg loss:{:4f} ".format(agent.avg_loss / float(step))
    log += "time:{:.2f}".format(time.time()-st)
    print(log)
    agent.avg_q_max, agent.avg_loss = 0, 0
    
    # 모델 저장
    if e % 5000 == 0:
        agent.model.save_weights("save_model/model", save_format="tf")
        bbox_agent.model.save_weights("save_model/bbox_model", save_format="tf")
    
    if is_pred:
        i = 0
        for p in used_bbox:
            p_loaded_idx = loaded_idx[i:i+len(p)]
            positions =  np.array(pred_pos)[i:i+len(p)]
            positions[:, :2] = positions[:, :2] /sc
            p_bbox = df_bbox.iloc[p[0]]
            p_x = df_p.iloc[p_loaded_idx].copy()
            p_x['pack_no'] = 'resid_'+str(pack_no)
            p_x['packing_rate_cbm'] = (p_x['cbmsum_per_mbox']).sum() / p_bbox['CBM']
            p_x['packing_rate_volume'] = (p_x['M_L']*p_x['M_B']*p_x['M_H']).sum() / (p_bbox['L']*p_bbox['B']*p_bbox['H'])
            p_x['x'] = positions[:, 0]
            p_x['y'] = positions[:, 1]
            p_x['z'] = positions[:, 2]
            pk = pk.append(p_x)

            pack_no += 1
            i += len(p)
    #break
#print('total training time', time.time()-st)
# if is_pred: pk.to_csv('data/pack/'+mi+'_resid_output.csv',index=False)

=====episode:    0| memory length:   20| epsilon:1.000| reward: 1, 0.232 (0.232)| avg loss:0.000000 time:0.58
=====episode:    1| memory length:   27| epsilon:1.000| reward: 1, 0.069 (0.069)| avg loss:0.000000 time:0.18
=====episode:    2| memory length:   56| epsilon:1.000| reward: 1, 0.397 (0.397)| avg loss:0.000000 time:1.18
=====episode:    3| memory length:   66| epsilon:1.000| reward: 1, 0.333 (0.333)| avg loss:0.000000 time:0.27
=====episode:    4| memory length:   77| epsilon:1.000| reward: 2, 0.363 (0.668, 0.057)| avg loss:0.000000 time:0.35
=====episode:    5| memory length:   93| epsilon:1.000| reward: 2, 0.442 (0.659, 0.224)| avg loss:0.000000 time:0.37
=====episode:    6| memory length:  134| epsilon:1.000| reward: 3, 0.763 (0.750, 0.764, 0.776)| avg loss:0.000000 time:2.00
=====episode:    7| memory length:  145| epsilon:1.000| reward: 1, 0.684 (0.684)| avg loss:0.000000 time:0.32
=====episode:    8| memory length:  225| epsilon:1.000| reward: 6, 0.627 (0.806, 0.750, 0.77

=====episode:   72| memory length: 1000| epsilon:0.997| reward: 2, 0.328 (0.558, 0.099)| avg loss:3.851011 time:2.66
=====episode:   73| memory length: 1000| epsilon:0.997| reward: 1, 0.255 (0.255)| avg loss:3.880498 time:0.68
=====episode:   74| memory length: 1000| epsilon:0.996| reward: 4, 0.676 (0.748, 0.530, 0.864, 0.563)| avg loss:3.840013 time:8.24
=====episode:   75| memory length: 1000| epsilon:0.996| reward: 3, 0.443 (0.706, 0.311, 0.312)| avg loss:3.797408 time:3.63
=====episode:   76| memory length: 1000| epsilon:0.996| reward: 3, 0.780 (0.813, 0.884, 0.644)| avg loss:3.769547 time:6.88
=====episode:   77| memory length: 1000| epsilon:0.996| reward: 4, 0.646 (0.766, 0.826, 0.785, 0.206)| avg loss:3.766709 time:9.62
=====episode:   78| memory length: 1000| epsilon:0.996| reward: 1, 0.491 (0.491)| avg loss:3.805163 time:0.70
=====episode:   79| memory length: 1000| epsilon:0.996| reward: 2, 0.527 (0.827, 0.228)| avg loss:3.797442 time:4.37
=====episode:   80| memory length: 1

=====episode:  144| memory length: 1000| epsilon:0.989| reward: 2, 0.519 (0.635, 0.404)| avg loss:3.641107 time:3.87
=====episode:  145| memory length: 1000| epsilon:0.989| reward: 2, 0.365 (0.580, 0.151)| avg loss:3.660031 time:1.21
=====episode:  146| memory length: 1000| epsilon:0.989| reward: 2, 0.473 (0.915, 0.030)| avg loss:3.633762 time:1.43
=====episode:  147| memory length: 1000| epsilon:0.988| reward: 1, 0.240 (0.240)| avg loss:3.599608 time:0.38
=====episode:  148| memory length: 1000| epsilon:0.988| reward: 4, 0.592 (0.928, 0.654, 0.693, 0.095)| avg loss:3.606647 time:5.25
=====episode:  149| memory length: 1000| epsilon:0.988| reward: 2, 0.429 (0.707, 0.151)| avg loss:3.642505 time:3.38
=====episode:  150| memory length: 1000| epsilon:0.988| reward: 1, 0.536 (0.536)| avg loss:3.629599 time:4.91
=====episode:  151| memory length: 1000| epsilon:0.988| reward: 1, 0.228 (0.228)| avg loss:3.633028 time:2.60
=====episode:  152| memory length: 1000| epsilon:0.988| reward: 2, 0.54

=====episode:  216| memory length: 1000| epsilon:0.981| reward: 1, 0.580 (0.580)| avg loss:3.435691 time:4.30
=====episode:  217| memory length: 1000| epsilon:0.981| reward: 1, 0.334 (0.334)| avg loss:3.474099 time:1.93
=====episode:  218| memory length: 1000| epsilon:0.981| reward: 1, 0.493 (0.493)| avg loss:3.423868 time:1.09
=====episode:  219| memory length: 1000| epsilon:0.981| reward: 4, 0.536 (0.761, 0.725, 0.500, 0.158)| avg loss:3.435453 time:6.26
=====episode:  220| memory length: 1000| epsilon:0.980| reward: 2, 0.572 (0.993, 0.150)| avg loss:3.415194 time:2.85
=====episode:  221| memory length: 1000| epsilon:0.980| reward: 2, 0.405 (0.700, 0.109)| avg loss:3.460496 time:1.79
=====episode:  222| memory length: 1000| epsilon:0.980| reward: 1, 0.309 (0.309)| avg loss:3.584052 time:0.39
=====episode:  223| memory length: 1000| epsilon:0.980| reward: 1, 0.307 (0.307)| avg loss:3.459008 time:2.53
=====episode:  224| memory length: 1000| epsilon:0.980| reward: 2, 0.645 (0.792, 0.49

=====episode:  288| memory length: 1000| epsilon:0.973| reward: 1, 0.933 (0.933)| avg loss:3.312894 time:4.99
=====episode:  289| memory length: 1000| epsilon:0.973| reward: 1, 0.183 (0.183)| avg loss:3.365929 time:1.34
=====episode:  290| memory length: 1000| epsilon:0.973| reward: 2, 0.346 (0.629, 0.063)| avg loss:3.375920 time:1.51
=====episode:  291| memory length: 1000| epsilon:0.973| reward: 1, 0.268 (0.268)| avg loss:3.362819 time:5.05
=====episode:  292| memory length: 1000| epsilon:0.972| reward: 2, 0.410 (0.452, 0.368)| avg loss:3.322693 time:1.68
=====episode:  293| memory length: 1000| epsilon:0.972| reward: 3, 0.658 (0.829, 0.876, 0.269)| avg loss:3.304228 time:2.57
=====episode:  294| memory length: 1000| epsilon:0.972| reward: 3, 0.827 (0.823, 1.002, 0.657)| avg loss:3.249333 time:12.12
=====episode:  295| memory length: 1000| epsilon:0.972| reward: 2, 0.613 (0.890, 0.335)| avg loss:3.260992 time:2.30
=====episode:  296| memory length: 1000| epsilon:0.972| reward: 1, 0.4

...

=====episode: 4157| memory length: 1000| epsilon:0.547| reward: 2, 0.465 (0.556, 0.374)| avg loss:2.091987 time:1.25
=====episode: 4158| memory length: 1000| epsilon:0.547| reward: 1, 0.250 (0.250)| avg loss:1.998861 time:0.79
=====episode: 4159| memory length: 1000| epsilon:0.547| reward: 3, 0.844 (0.834, 0.849, 0.848)| avg loss:1.987099 time:4.25
=====episode: 4160| memory length: 1000| epsilon:0.547| reward: 1, 0.208 (0.208)| avg loss:2.162149 time:0.94
=====episode: 4161| memory length: 1000| epsilon:0.547| reward: 2, 0.849 (0.755, 0.943)| avg loss:2.088836 time:7.86
=====episode: 4162| memory length: 1000| epsilon:0.547| reward: 2, 0.523 (0.630, 0.415)| avg loss:1.979630 time:1.21
=====episode: 4163| memory length: 1000| epsilon:0.547| reward: 2, 0.467 (0.804, 0.131)| avg loss:2.048625 time:3.87
=====episode: 4164| memory length: 1000| epsilon:0.547| reward: 1, 0.698 (0.698)| avg loss:2.073442 time:2.75
=====episode: 4165| memory length: 1000| epsilon:0.546| reward: 1, 0.275 (0.27

=====episode: 4228| memory length: 1000| epsilon:0.540| reward: 3, 0.546 (0.809, 0.777, 0.053)| avg loss:2.053361 time:7.43
=====episode: 4229| memory length: 1000| epsilon:0.539| reward: 2, 0.366 (0.536, 0.196)| avg loss:2.121051 time:2.09
=====episode: 4230| memory length: 1000| epsilon:0.539| reward: 2, 0.737 (0.848, 0.627)| avg loss:2.093183 time:5.02
=====episode: 4231| memory length: 1000| epsilon:0.539| reward: 2, 0.360 (0.471, 0.250)| avg loss:2.092603 time:1.29
=====episode: 4232| memory length: 1000| epsilon:0.539| reward: 2, 0.671 (0.741, 0.601)| avg loss:2.104822 time:7.36
=====episode: 4233| memory length: 1000| epsilon:0.539| reward: 2, 0.480 (0.892, 0.068)| avg loss:2.072455 time:6.08
=====episode: 4234| memory length: 1000| epsilon:0.539| reward: 2, 0.603 (0.721, 0.486)| avg loss:2.094083 time:2.46
=====episode: 4235| memory length: 1000| epsilon:0.539| reward: 1, 0.785 (0.785)| avg loss:2.054635 time:5.19
=====episode: 4236| memory length: 1000| epsilon:0.539| reward: 

=====episode: 4299| memory length: 1000| epsilon:0.532| reward: 1, 0.372 (0.372)| avg loss:2.119761 time:0.91
=====episode: 4300| memory length: 1000| epsilon:0.532| reward: 2, 0.361 (0.206, 0.516)| avg loss:2.064954 time:0.89
=====episode: 4301| memory length: 1000| epsilon:0.532| reward: 1, 0.771 (0.771)| avg loss:2.011152 time:5.29
=====episode: 4302| memory length: 1000| epsilon:0.531| reward: 3, 0.561 (0.814, 0.690, 0.179)| avg loss:1.964269 time:10.69
=====episode: 4303| memory length: 1000| epsilon:0.531| reward: 1, 0.508 (0.508)| avg loss:1.924228 time:1.80
=====episode: 4304| memory length: 1000| epsilon:0.531| reward: 1, 0.258 (0.258)| avg loss:1.994264 time:1.49
=====episode: 4305| memory length: 1000| epsilon:0.531| reward: 1, 0.249 (0.249)| avg loss:1.880046 time:0.68
=====episode: 4306| memory length: 1000| epsilon:0.531| reward: 1, 0.562 (0.562)| avg loss:2.023452 time:1.62
=====episode: 4307| memory length: 1000| epsilon:0.531| reward: 2, 0.447 (0.851, 0.042)| avg loss:

=====episode: 4370| memory length: 1000| epsilon:0.524| reward: 1, 0.080 (0.080)| avg loss:2.167392 time:0.71
=====episode: 4371| memory length: 1000| epsilon:0.524| reward: 3, 0.591 (0.702, 0.708, 0.364)| avg loss:2.132394 time:12.33
=====episode: 4372| memory length: 1000| epsilon:0.524| reward: 1, 0.062 (0.062)| avg loss:2.248885 time:0.22
=====episode: 4373| memory length: 1000| epsilon:0.524| reward: 2, 0.481 (0.596, 0.367)| avg loss:2.209748 time:0.68
=====episode: 4374| memory length: 1000| epsilon:0.523| reward: 4, 0.519 (0.625, 0.600, 0.793, 0.057)| avg loss:2.148504 time:8.17
=====episode: 4375| memory length: 1000| epsilon:0.523| reward: 2, 0.546 (0.467, 0.625)| avg loss:2.144069 time:3.22
=====episode: 4376| memory length: 1000| epsilon:0.523| reward: 1, 0.777 (0.777)| avg loss:2.156539 time:4.15
=====episode: 4377| memory length: 1000| epsilon:0.523| reward: 2, 0.535 (0.744, 0.326)| avg loss:2.157318 time:3.68
=====episode: 4378| memory length: 1000| epsilon:0.523| reward:

=====episode: 4442| memory length: 1000| epsilon:0.516| reward: 1, 0.734 (0.734)| avg loss:2.084577 time:1.17
=====episode: 4443| memory length: 1000| epsilon:0.516| reward: 2, 0.541 (0.893, 0.190)| avg loss:2.091876 time:14.43
=====episode: 4444| memory length: 1000| epsilon:0.516| reward: 1, 0.739 (0.739)| avg loss:2.113053 time:3.47
=====episode: 4445| memory length: 1000| epsilon:0.516| reward: 1, 0.758 (0.758)| avg loss:2.166997 time:2.80
=====episode: 4446| memory length: 1000| epsilon:0.516| reward: 2, 0.497 (0.897, 0.097)| avg loss:2.155313 time:4.31
=====episode: 4447| memory length: 1000| epsilon:0.515| reward: 2, 0.424 (0.757, 0.091)| avg loss:2.115849 time:1.81
=====episode: 4448| memory length: 1000| epsilon:0.515| reward: 2, 0.381 (0.729, 0.034)| avg loss:2.099606 time:6.06
=====episode: 4449| memory length: 1000| epsilon:0.515| reward: 2, 0.351 (0.644, 0.057)| avg loss:2.092192 time:2.03
=====episode: 4450| memory length: 1000| epsilon:0.515| reward: 5, 0.431 (0.283, 0.4